In [37]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]) + '/src')
from ast import literal_eval
from datasets.ie_hyperion_dataset import find_word_bounds, clean_text

df = pd.read_csv('../data/processed/splitted_union/ie_s2_hyperion_test.csv', converters={'Stralci': literal_eval, 'Repertori': literal_eval})
#df = df.head(50)
df['Testo'] = df['Testo'].map(clean_text)
df['Stralci'] = df['Stralci'].map(lambda x: [clean_text(s) for s in x])
df['Bounds'] = df.apply(lambda x: find_word_bounds(x['Stralci'], x['Testo']), axis=1).values.tolist()

In [34]:
len(df.index)

3066

In [38]:
from models.nltk_segmenter import NLTKSegmenter
from trainers.bert_seg_trainer import *


nltk_seg = NLTKSegmenter()
df['Stralci_predetti'] = df['Testo'].map(nltk_seg.predict).values.tolist()
df['Bounds_predetti'] = df.apply(lambda x: find_word_bounds(x['Stralci_predetti'], x['Testo']), axis=1).values.tolist()

df['Bounds_predetti_norm'] = df.apply(lambda x: normalize_bounds_by_repertoire(x['Bounds_predetti'], x), axis=1).values.tolist()

In [39]:
from trainers.bert_seg_trainer import *
from datasets.ie_hyperion_dataset import IEHyperionDataset

test_dataset = IEHyperionDataset(df, 'dbmdz/bert-base-italian-xxl-uncased')

metrics = compute_metrics(df['Bounds_predetti'].to_list(), test_dataset)
metrics

{'windowdiff': 0.5377331087580638,
 'ghd': 69.07510917030568,
 'pk': 0.3048955724237861,
 'iou': 0.5091148174116299}

In [40]:
norm_metrics = compute_metrics(df['Bounds_predetti_norm'].to_list(), test_dataset)
norm_metrics

{'windowdiff': 0.32240403731538053,
 'ghd': 27.031441048034935,
 'pk': 0.20217541539007025,
 'iou': 0.878052741961476}